In [ ]:
from __future__ import print_function

import math
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Layer, Dense, Dropout, Input, LeakyReLU
from keras.layers.core import Activation
from keras.optimizers import RMSprop
from keras.initializers import Constant, glorot_normal
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from sklearn.model_selection import KFold
import numpy as np
import scipy.io as spio
import random
import sys
import pandas as pd
import os
from sklearn.metrics import cohen_kappa_score,roc_auc_score,accuracy_score,average_precision_score
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# set parameters

bins=10
batch_size=64
start_temp=10.0
min_temp=0.1
lossWeights = {"recon":1, "classacc":100}
losses = {"recon": "mean_squared_error", "classacc": "binary_crossentropy",}
cmi=0
mi=0

In [ ]:
## Definition of FSNet method

def calc_MI(X,Y,bins):
    c_XY = np.histogram2d(X,Y,bins)[0]
    c_X = np.histogram(X,bins)[0]
    c_Y = np.histogram(Y,bins)[0]
    H_X = shan_entropy(c_X)
    H_Y = shan_entropy(c_Y)
    H_XY = shan_entropy(c_XY)
    mi1 = H_X + H_Y - H_XY
    return mi1

def shan_entropy(c):
    c_normalized = c / float(np.sum(c))
    c_normalized = c_normalized[np.nonzero(c_normalized)]
    H = -sum(c_normalized* np.log2(c_normalized))  
    return H

def MI(S):
    bins = 10
    n = S.shape[1]
    mis=0
    count=0
    for ix in np.arange(n):
        for jx in np.arange(ix+1,n):
            mis = mis+calc_MI(S[:,ix], S[:,jx], bins)
            count=count+1
    mis=mis/count
    return mis


class tinyLayerE(Layer):
    def __init__(self, output_dim, u, bins, start_temp=10.0, min_temp=0.1, alpha=0.99999, **kwargs):
        self.output_dim=output_dim
        self.u=K.constant(u)
        self.start_temp = start_temp
        self.min_temp = K.constant(min_temp)
        self.alpha = K.constant(alpha)
        super(tinyLayerE, self).__init__(**kwargs)

    def build(self,input_shape):
        self.temp = self.add_weight(name = 'temp', shape = [], initializer = Constant(self.start_temp), trainable = False)
        
        self.tinyW=self.add_weight(name='tinyW', shape=(bins,self.output_dim), initializer='uniform', trainable=True)
        super(tinyLayerE,self).build(input_shape)

    def call(self, X, training = None):
        
        al=K.softmax(K.dot(self.u,self.tinyW))
        al=K.transpose(al) 
        logits=K.log(10*K.maximum(K.minimum(al,0.9999999),K.epsilon()))
        uniform = K.random_uniform(logits.shape, K.epsilon(), 1.0)
        gumbel = -K.log(-K.log(uniform))
        temp = K.update(self.temp, K.maximum(self.min_temp, self.temp * self.alpha))
        noisy_logits = (logits+gumbel) / temp
        samples = K.softmax(noisy_logits)
        discrete_logits = K.one_hot(K.argmax(logits), logits.shape[1])
        self.logits=samples
        dl = np.zeros(self.logits.shape)
        p = K.get_value(self.logits)
        
        for i in range(dl.shape[0]):
            ind = np.argmax(p, axis=None)
            x=ind//dl.shape[1]
            y=ind%dl.shape[1]
            dl[x][y]=1
            p[x]=-np.ones(dl.shape[1])
            p[:,y]=-np.ones(dl.shape[0])
            discrete_logits = K.one_hot(K.argmax(K.variable(dl)), dl.shape[1])
        
        self.selections = K.in_train_phase(samples, discrete_logits, training)
        Y = K.dot(X, K.transpose(self.selections))
        
        return Y

    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

class tinyLayerD(Layer):
    
    def __init__(self, output_dim, u, bins, **kwargs):
        self.output_dim=output_dim
        self.u=K.constant(u)
        super(tinyLayerD, self).__init__(**kwargs)
  
    def build(self,input_shape):
        self.tinyW=self.add_weight(name='tinyW', shape=(bins, input_shape[1]), initializer='uniform', trainable=True)
        super(tinyLayerD,self).build(input_shape)

    def call(self, x):
        weights=K.transpose(K.tanh(K.dot(self.u,self.tinyW)))
        return K.dot(x,weights)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


In [ ]:
def set_data(path,data_name,k_feat):
    
    data = np.genfromtxt(path+ data_name + str(k_feat) + 'feat.csv',delimiter=',')

    X = data[:,:-1]
    y = data[:,-1]
    
    return X,y

def matches(best_feat,non_zero):
    match = []
    for i in range(len(best_feat)):
        for j in range(len(non_zero)):
            if best_feat[i] == non_zero[j]:
                match.append(best_feat[i])
    return match

def get_features(model_names,n_split,k):                     
    
    best_feat = np.zeros((n_split,k))
       
    for i in range(n_split):
            
        imp = spio.loadmat('/Users/utente/Documents/università/tesi - confronto FS/analisi 2-512 features/FSNet/'+model_names[1] + "_fold_" + str(i) + "_indices.mat")
        best_feat[i] = imp['indices']
        
    return best_feat


def cv_fsnet(X,Y,num_exp,k_feat,l,h_size,num_epochs,dataDir,model_name = 'fs'):
    
    # num_exp: number of folds for KFold cross val
    # l: learning rate
    # k_feat: Number of features to select
    # h_size: neurons per layer
    
    opt=RMSprop(learning_rate=l)    
    
    kf = KFold(n_splits=num_exp)
    
    # Normalization to N(0,1)
    
    
    X=np.delete(X,np.where(np.std(X,axis=0)==0),axis=1)
    for i in range(X.shape[1]):
        if np.max(X[:,i])!=0:
            X[:,i]=X[:,i]/np.max(np.absolute(X[:,i]))
            mu_Xi=np.mean(X[:,i])
            std_Xi=np.std(X[:,i])
            X[:,i]=X[:,i]-mu_Xi
            if std_Xi!=0:
                X[:,i]=X[:,i]/std_Xi
  
    aucc = []
    auprc = []
    fold = 0
    for tr_idx, te_idx in kf.split(X):
        
        x_train, x_test = X[tr_idx], X[te_idx]
        y_train, y_test = Y[tr_idx], Y[te_idx]
        
        u_train=np.zeros([x_train.shape[1],bins],dtype=float)
        for i in range(0,x_train.shape[1]):
            hist=np.histogram(x_train[:,i],bins)
            for j in range(0,bins):
                u_train[i,j]=hist[0][j]*0.5*(hist[1][j]+hist[1][j+1])

        steps_per_epoch = (len(x_train) + batch_size - 1) // batch_size
        alpha = math.exp(math.log(min_temp / start_temp) / (num_epochs * steps_per_epoch))
       
  ################################
  # FsNet
  ################################

        inp1=Input(shape=(x_train.shape[1],))
        x=tinyLayerE(k_feat,u_train,bins,start_temp, min_temp, alpha, name = 'tinyLayerE')(inp1)
        
        x = Dense(int(h_size))(x)
        x = LeakyReLU(0.2)(x)
        x = Dense(int(h_size))(x)
        x = LeakyReLU(0.2)(x)
        x = Dense(int(h_size))(x)
        x = LeakyReLU(0.2)(x)
        
        
        x1 = Dense(int(h_size))(x)
        x1 = LeakyReLU(0.2)(x1)
        x1 = Dense(int(h_size))(x1)
        x1 = LeakyReLU(0.2)(x1)
        x1 = Dense(int(h_size))(x1)
        x1 = LeakyReLU(0.2)(x1)
        
        
        x1 = tinyLayerD(x_train.shape[1],u_train,bins,name = 'recon')(x1)
        x2 = Dense(1,activation="sigmoid", name="classacc")(x)
        model = Model(inputs=inp1, outputs=[x1, x2])
        callback_early = EarlyStopping(monitor='val_classacc_loss', patience=100,restore_best_weights=True)
        #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.33,patience=3, min_lr=0.00001)
        model.compile(optimizer=opt, loss=losses, loss_weights=lossWeights, metrics=["accuracy","mse"])
        
        model.summary()
       
        history = model.fit(x_train, {"recon": x_train, "classacc": y_train},validation_split=0.2 ,epochs=num_epochs, callbacks=[callback_early])#validation_data=(x_test, {"recon": x_test, "classacc": y_test})
        

        x_pred,y_pred = model.predict(x_test)
        
        aucc.append(roc_auc_score(y_test,y_pred))
        auprc.append(average_precision_score(y_test, y_pred))
       
        outputDir = os.path.join( dataDir, 'FSNet')
        try:
            os.stat(outputDir)
        except:
            os.mkdir(outputDir)

        
        with open(os.path.join(outputDir, model_name + '_fold_'+ str(i) +'_ypredproba.csv'), "a+") as myfile:
            myfile.write(','.join([str(x) for x in y_pred.flatten()]) + '\n')
            
        probabilities = K.get_value(K.softmax(model.get_layer('tinyLayerE').logits))
        dl=np.zeros(model.get_layer('tinyLayerE').logits.shape)
        p=K.get_value(model.get_layer('tinyLayerE').logits)
        for j in range(dl.shape[0]):
            ind=np.argmax(p,axis=None)
            x=ind//dl.shape[1]
            y=ind%dl.shape[1]
            dl[x][y]=1
            p[x]=-np.ones(dl.shape[1])
            p[:,y]=-np.ones(dl.shape[0])

        indices = K.get_value(K.argmax(dl))

        spio.savemat(dataDir+'/FSNet/'+model_name + "_fold_" + str(fold) + '_indices.mat', {'indices': indices})
        fold += 1

    with open(os.path.join(outputDir, 'cv_cancelout_' + model_name + '_auc.csv'), "a+") as myfile:
        myfile.write(str(aucc))
    with open(os.path.join(outputDir, 'cv_cancelout_' + model_name + '_auc.csv'), "a+") as myfile:
        myfile.write(str(auprc))
        
    return np.mean(aucc), np.mean(auprc) ,  np.var(aucc), np.var(auprc)     


In [ ]:
num_epochs = 1000
cacc=np.zeros(num_epochs)
acc=np.zeros(num_epochs)
closs=np.zeros(num_epochs)
loss=np.zeros(num_epochs)

dataDir = '/Users/utente/Documents/università/tesi - confronto FS/analisi 2-64 features'

path='./data/'
data_name='ring-xor-sum_1000samples-' #RING: 'ring_1000samples-'; XOR: 'xor_1000samples-'; RING+XOR: 'ring+xor_1000samples-'; RING+XOR+SUM: 'ring-xor-sum_1000samples-'
h_size = 64
splits = 6
lr = 0.005    
K_feat = 6
tot_feats = [6,8,16,32,64,128,256,512]
dataset = 'ring-xor-sum_K_'

cv_auc = []
var_auc = []
cv_auprc = []
var_auprc = []

for i in tot_feats:
    
    X,y = set_data(path,data_name,i)

    auccc,auprc,vauccc,vauprc = cv_fsnet(X,y,splits,K_feat,lr,h_size,num_epochs,dataDir,dataset+str(i)+'_fs')

    var_auc.append(vauccc)
    cv_auc.append(auccc)
    var_auprc.append(vauprc)
    cv_auprc.append(auprc)

In [ ]:
name0 = 'Fsnet'
name1 = '_fs'
tot_feats = [6,8,16,32,64,128,256,512]

feat_res = []
feat_res_2k = []

# Best K features

for i in tot_feats:
    feat_res.append(get_features([name0,dataset+str(i)+name1],splits,K_feat))
    
m = np.zeros((len(tot_feats),splits)) #rows=ring2,ring4,ring8... columns=fold0,fold1,fold2...
for j in range(len(tot_feats)):
    for k in range(splits):
        m[j,k] =len(matches(feat_res[j][k],np.arange(K_feat)))

# best K feat averaged on the 6 folds
cv_feat = np.mean(m,axis=1) 
var_feat = np.var(m,axis=1)

In [ ]:
# Save the results

df = pd.DataFrame([cv_auc,var_auc,cv_auprc,var_auprc,cv_feat,var_feat])
df1 = df.T
df1
df1.to_excel("Acc-Cohen-Auc-Auprc_FSNET_K_RING-XOR-SUM.xlsx")

In [ ]:
# Run the model selecting 2k features

K_feat = 12
tot_feats = [16,32,64,128,256,512]
dataset = 'ring-xor-sum_2K_'

cv_auc = []
var_auc = []
cv_auprc = []
var_auprc = []

for i in tot_feats:
    
    X,y = set_data(path,data_name,i)

    auccc,auprc,vauccc,vauprc = cv_fsnet(X,y,splits,K_feat,lr,h_size,num_epochs,dataDir,dataset+str(i)+'_fs')

    var_auc.append(vauccc)
    cv_auc.append(auccc)
    var_auprc.append(vauprc)
    cv_auprc.append(auprc)

In [ ]:

# Best 2K features

for i in tot_feats:
        feat_res_2k.append(get_features([name0,dataset+str(i)+name1],splits,int(K_feat)))

m = np.zeros((len(tot_feats),splits)) #rows=ring2,ring4,ring8... columns=fold0,fold1,fold2...
for j in range(len(tot_feats)):
    for k in range(splits):
        
        m[j,k] =len(matches(feat_res_2k[j][k],np.arange(int(K_feat/2))))
        
# best 2K feat averaged on the 6 folds
cv_feat_2k = np.mean(m,axis=1)
var_feat_2k = np.var(m,axis=1)

In [ ]:
# Save the results

df = pd.DataFrame([cv_auc,var_auc,cv_auprc,var_auprc,cv_feat_2k,var_feat_2k])
df1 = df.T
df1
df1.to_excel("Acc-Cohen-Auc-Auprc_FSNET_2K_RING-XOR-SUM.xlsx")